### Libraries installation

In [1]:
! pip uninstall numpy

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/numpy-config
    /usr/local/lib/python3.11/dist-packages/numpy-2.0.2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1-0352e75f.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99-934c22de.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libscipy_openblas64_-99b71e71.so
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? Y
  Successfully uninstalled numpy-2.0.2


In [2]:
! pip install numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 75.8 MB/s eta 0:00:00


In [1]:
import numpy as np
print(np.__version__)

1.26.4


If for some reason it's not equal to 1.26.4, please restart the notebook. It's important to run the catboost classification.

It seems that current version of catboost is not compatible with numpy 2.x.

In [2]:
! pip install pip-system-certs striprtf catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.0 MB/s eta 0:00:00


### Dataset creation

In [3]:
from dataset_preparation import get_all_data

In [4]:
url = "https://people.rc.rit.edu/bsm9339/corpora/squinky_corpus/mturk_merged.csv" # squinky dataset link
filenames = ['formal_sentences.rtf', 'informal_sentences.rtf', 'middle_sentences.rtf'] # gpt responses

all_data = get_all_data(url, filenames)


### Models

DeBerta model creation:

In [5]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch


model_name = 's-nlp/deberta-large-formality-ranker'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
deberta_tokenizer = AutoTokenizer.from_pretrained(model_name)
deberta = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/790 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

XLM-Roberta-based classifier creation:

In [6]:
from transformers import XLMRobertaTokenizerFast, XLMRobertaForSequenceClassification


roberta_tokenizer = XLMRobertaTokenizerFast.from_pretrained('s-nlp/xlmr_formality_classifier')
roberta = XLMRobertaForSequenceClassification.from_pretrained('s-nlp/xlmr_formality_classifier').to(device)

tokenizer_config.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/896 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

### Catboost

Catboost datasets creation:

In [7]:
from dataset_preparation import catboost_data_preparation

X_train, X_test, y_train, y_test = catboost_data_preparation(all_data)

Training and inference step:

In [8]:
from catboost import CatBoostClassifier

catboost = CatBoostClassifier(verbose=0)
catboost.fit(X_train, y_train)

Prediction:

In [9]:
y_pred = catboost.predict(X_test)

### Metrics




Quality estimation. First you have to create a dataloader, then simply run the evaluation process. It prints evaluation report.

In [10]:
input_data = all_data['sentence'].to_list()
labels = all_data['class'].to_list()


listed_data = list(zip(input_data, labels))

In [11]:
from torch.utils.data import DataLoader

val_loader = DataLoader(listed_data, shuffle=False, batch_size = 50)

In [12]:
from evaluation import evaluate_model

In [13]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [14]:
evaluate_model(deberta, deberta_tokenizer,  val_loader, device)

100%|██████████| 156/156 [00:38<00:00,  4.06it/s]


              precision    recall  f1-score   support

      formal       0.80      0.77      0.78      4056
    informal       0.76      0.79      0.78      3744

    accuracy                           0.78      7800
   macro avg       0.78      0.78      0.78      7800
weighted avg       0.78      0.78      0.78      7800



In [15]:
evaluate_model(roberta, roberta_tokenizer, val_loader, device)

100%|██████████| 156/156 [00:10<00:00, 14.77it/s]


              precision    recall  f1-score   support

      formal       0.76      0.50      0.60      4056
    informal       0.61      0.83      0.70      3744

    accuracy                           0.66      7800
   macro avg       0.69      0.67      0.65      7800
weighted avg       0.69      0.66      0.65      7800



To estimate CatboosrClassifier one has to just run `classification_report` function from `sklearn`

In [16]:
from sklearn.metrics import classification_report

print(classification_report(y_test.to_list(), y_pred, target_names=['formal', 'informal']))

              precision    recall  f1-score   support

      formal       0.82      0.83      0.83       777
    informal       0.83      0.82      0.83       780

    accuracy                           0.83      1557
   macro avg       0.83      0.83      0.83      1557
weighted avg       0.83      0.83      0.83      1557

